<h1 align="center">  Object Detection </h1>
Object Detection task with fcos model. 

Mount Google Drive and go the the fcos directory. Suppose we have uploaded `ai_training` to Google Drive.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/ai_training/detection/fcos/

/content/drive/MyDrive/ai_training/detection/fcos


# Prerequisites
- Python >= 3.6

# Installation
To install the dependencies, run

In [3]:
!pip install -r requirements-colab.txt
!python setup.py build_ext --inplace

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-3pvk1zin
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-3pvk1zin
     |████████████████████████████████| 317kB 7.5MB/s 
     |████████████████████████████████| 30.6MB 96kB/s 
     |████████████████████████████████| 25.0MB 123kB/s 
     |████████████████████████████████| 2.9MB 44.8MB/s 
     |████████████████████████████████| 645kB 36.5MB/s 
     |████████████████████████████████| 4.8MB 39.5MB/s 
     |████████████████████████████████| 4.1MB 28.9MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20486 sha256=5972a496d0afaf240ff78ef9458cd88eca706c7e0055b9b8c8e86b1d25c81897
  Stored in directory: /root/.cache/pip/wheels/5f/09/a5/497a30fd9ad9964e98a1254d1e164bcd1b8a5eda36197ecb3c
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=263915

In [4]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.test.is_gpu_available()) # should get 'True'
print(tf.__version__) # should get '1.15.2'

TensorFlow 1.x selected.
True
1.15.2


# Dataset & Preparation
For this tutorial, we will use COCO128 dataset, located under `detection/yolov5/coco128`. Here is the data yaml file for COCO128:


In [11]:
!cat data/coco128.yaml

train: ../yolov5/coco128/images/train2017  # 128 images
val: ../yolov5/coco128/images/train2017  # 128 images

# number of classes
nc: 80

# type of dataset
dataset_type: csv

# class names
names: ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven',

# Train with COCO128

Let's finetune a pretrained model on COCO128 custom dataset (located under yolov5 folder). The pretrained model we used here is the model with backbone Darknet53s, pan FPN type, trained on COCO dataset. We download the pretrained model from [Model_Zoo](https://github.com/kneron/Model_Zoo/tree/main/detection/fcos). Since COCO128 is small, we choose to freeze the pretrained model backbone. Execute commands:

In [ ]:
!wget https://raw.githubusercontent.com/kneron/Model_Zoo/main/detection/fcos/coco_yolov5_pan_3_11_1.9920_0.4832.h5

--2021-05-20 23:44:26--  https://raw.githubusercontent.com/kneron/Model_Zoo/main/detection/fcos/coco_yolov5_pan_3_11_1.9920_0.4832.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34583584 (33M) [application/octet-stream]
Saving to: ‘coco_yolov5_pan_3_11_1.9920_0.4832.h5’

coco_yolov5_pan_3_1 100%[===================>]  32.98M  63.1MB/s    in 0.5s    

2021-05-20 23:44:27 (63.1 MB/s) - ‘coco_yolov5_pan_3_11_1.9920_0.4832.h5’ saved [34583584/34583584]



In [6]:
!python train.py --backbone darknet53s --fpn pan --snapshot coco_yolov5_pan_3_11_1.9920_0.4832.h5 --freeze-backbone --batch-size 8 --gpu 0 --steps 5 --epochs 2 --snapshot-path snapshots/exp --data data/coco128.yaml


Using TensorFlow backend.
{'data': 'data/coco128.yaml', 'snapshot': 'coco_yolov5_pan_3_11_1.9920_0.4832.h5', 'backbone': 'darknet53s', 'fpn': 'pan', 'reg_func': 'linear', 'stage': 3, 'head_type': 'simple', 'centerness_pos': 'reg', 'batch_size': 8, 'gpu': '0', 'epochs': 2, 'steps': 5, 'lr': 0.0001, 'snapshot_path': 'snapshots/exp', 'freeze_backbone': True, 'input_size': 512, 'compute_val_loss': False}


2021-05-29 01:34:28.167297: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-05-29 01:34:28.167471: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x562e6264b100 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-05-29 01:34:28.167501: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-05-29 01:34:28.169166: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-05-29 01:34

As we can see from the messages, the traininng losses and time are printed and the validation mAP were reported in each epoch.  The trained model will be saved to `snapshots/exp` folder. 

# Inference 
In this section, we will go through an example of using a trained network for inference. That is, we'll pass an image into the network and detect and classify the object in the image. We will use the function `inference.py` that takes an image and a model, then returns the detection information. The output format is a list of list, [[l,t,w,h,score,class_id], [l,t,w,h,score,class_id] ...]. We can also draw the bbox on the image if the save path is given. 

Let's run pretrained network on a screenshot from a movie, with the following code:

In [7]:
!python inference.py --snapshot snapshots/exp/csv_darknet53s_pan_3_01.h5 --score-thres 0.6 --gpu 0 --class-id-path utils/coco_id_class_map.json --img-path tutorial/demo/fcos_demo.jpg --save-path tutorial/demo/out.jpg

Using TensorFlow backend.
{'img_path': 'tutorial/demo/fcos_demo.jpg', 'class_id_path': 'utils/coco_id_class_map.json', 'gpu': 0, 'snapshot': 'snapshots/exp/csv_darknet53s_pan_3_01.h5', 'input_shape': [512, 512], 'max_objects': 100, 'score_thres': 0.6, 'iou_thres': 0.5, 'nms': 1, 'save_path': 'tutorial/demo/out.jpg', 'save_preds_path': None}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2021-05-29 01:36:20.958229: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-05-29 01:36:20.963715: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-29 01:36:20.964291: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2021-05-29 01:

We get the inference result saved in a json file located under the same folder as the input image, `./tutorial/demo/fcos_demo_preds.json`.

In [8]:
!cat ./tutorial/demo/fcos_demo_preds.json

{"img_path": "tutorial/demo/fcos_demo.jpg", "0_0": [[909.0585994720459, 158.15237045288086, 305.4385471343994, 764.1001510620117, 0.7487878203392029, 0.0], [641.4619374275208, 202.17178344726562, 264.6464467048645, 726.476469039917, 0.7456897497177124, 0.0]]}

# Convert to ONNX

Pull the latest [ONNX converter](https://github.com/kneron/ONNX_Convertor/tree/master/keras-onnx) from github. You may read the latest document from Github for converting ONNX model. Execute commands in the folder `ONNX_Convertor/keras-onnx`:
(reference: https://github.com/kneron/ONNX_Convertor/tree/master/keras-onnx)



In [9]:
!git clone https://github.com/kneron/ONNX_Convertor.git

Cloning into 'ONNX_Convertor'...
remote: Enumerating objects: 1174, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 1174 (delta 84), reused 78 (delta 39), pack-reused 1020
Receiving objects: 100% (1174/1174), 5.85 MiB | 15.10 MiB/s, done.
Resolving deltas: 100% (784/784), done.
Checking out files: 100% (225/225), done.


In [10]:
!python ONNX_Convertor/keras-onnx/generate_onnx.py -o snapshots/exp/csv_darknet53s_pan_3_01_converted.onnx snapshots/exp/csv_darknet53s_pan_3_01.h5


Using TensorFlow backend.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2021-05-29 01:36:44.716898: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-05-29 01:36:44.722177: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-29 01:36:44.722732: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2021-05-29 01:36:44.723007: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-05-29 01:36:44.724479: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-05-29 01:36:44.732524: I tensorflow/

We get the onnx model `csv_darknet53s_pan_3_01_converted.onnx` under `snapshots/exp` folder.


